<a href="https://colab.research.google.com/github/pedrobcavalcante/Backtest-trading-strategies/blob/main/Gerador_de_Par%C3%A2metros_para_estrat%C3%A9gias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [12]:
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import os.path
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
backtesting.set_bokeh_output(notebook=False)


In [13]:
from backtesting.test import SMA, GOOG
# GOOG.to_csv("data/GOOG.csv")


In [14]:
def fixData(data):
    columns = ['open_time', 'Open', 'High', 'Low', 'Close', 'Volume',
               'close_time', 'qav', 'num_trades',
               'taker_base_vol', 'taker_quote_vol', 'ignore']
    df = pd.DataFrame(data, columns=columns)
    df.index = [dt.datetime.fromtimestamp(x/1000.0) for x in df.close_time]
    df.sort_values(["open_time"], ascending=False)
    df = df.astype('double')
    df['Volume'] = df['Volume'].astype('int')
    df = df.drop(['qav', 'num_trades', 'taker_base_vol',
                 'taker_quote_vol', 'ignore'], axis=1)
    return df


In [15]:
def get_bars(symbol, interval='1h'):
    root_url = 'https://api.binance.com/api/v1/klines'
    filename = "data/"+symbol+"_"+interval+".csv"
    url = root_url + '?symbol=' + symbol + '&interval=' + interval+"&limit=1001"
    print(url)
    data = json.loads(requests.get(url).text)
    df = fixData(data)
#    df.to_csv(filename, sep=';')
    return df


In [16]:
dataframe = get_bars("BTCUSDT", "1d")
dataframe


https://api.binance.com/api/v1/klines?symbol=BTCUSDT&interval=1d&limit=1001


,open_time,Open,High,Low,Close,Volume,close_time
2019-01-30 20:59:59.999,1.548806e+12,3410.04,3478.00,3387.10,3458.18,23968,1.548893e+12
2019-01-31 20:59:59.999,1.548893e+12,3457.50,3489.20,3418.80,3434.10,29607,1.548979e+12
2019-02-01 20:59:59.999,1.548979e+12,3434.10,3488.00,3401.20,3462.07,25260,1.549066e+12
2019-02-02 20:59:59.999,1.549066e+12,3462.20,3526.40,3440.29,3504.77,17920,1.549152e+12
2019-02-03 20:59:59.999,1.549152e+12,3504.06,3511.09,3426.00,3458.11,19867,1.549238e+12
...,...,...,...,...,...,...,...
2021-10-21 20:59:59.999,1.634774e+12,66001.40,66639.74,62000.00,62193.15,68538,1.634861e+12
2021-10-22 20:59:59.999,1.634861e+12,62193.15,63732.39,60000.00,60688.22,52119,1.634947e+12
2021-10-23 20:59:59.999,1.634947e+12,60688.23,61747.64,59562.15,61286.75,27626,1.635034e+12
2021-10-24 20:59:59.999,1.635034e+12,61286.75,61500.00,59510.63,60852.22,31226,1.635120e+12


In [17]:
class SmaCross(Strategy):
    n1 = 0
    n2 = 0

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()

    def setInitialParams(self, valor1, valor2):
        self.n1 = valor1
        self.n2 = valor2


In [18]:
def gerarParametros(dataframe, name, myStrategy, tamanho):
    maiorRetorno = {}
    maoirAcerto = {}
    BoasEstratégiasParaSalvar = []
    for s1 in range(tamanho):
        s1 += 1
        print("\ntestando nos valor ", s1, end=" ")
        for s2 in range(tamanho):
            s2 += 1
            # print(",", s2, end="")
            try:
                SmaCross.setInitialParams(SmaCross, s1, s2)
                bt = Backtest(dataframe, myStrategy,
                              cash=100000000, exclusive_orders=True)
                output = bt.run()
                if maiorRetorno == {}:
                    maiorRetorno = {"sma1": s1, "sma2": s2, "output": output}
                if maoirAcerto == {}:
                    maoirAcerto = {"sma1": s1, "sma2": s2, "output": output}
                if output['Return [%]'] > maiorRetorno["output"]['Return [%]'] and output['# Trades'] > 9:
                    maiorRetorno = {"sma1": s1, "sma2": s2, "output": output}
                if output['Return [%]'] > 0 and output['# Trades'] > 9:
                    BoasEstratégiasParaSalvar.append(
                        {"sma1": s1, "sma2": s2, "output": output})
        #         if output['Win Rate [%]'] > maiorRetorno["output"]['Win Rate [%]'] and output['# Trades'] > 9:
        #             maoirAcerto = {"sma1": s1, "sma2":s2, "output":output}
            except Exception as e:
                print(e)
    if not os.path.exists('estrategy results'):
        os.makedirs('estrategy results')
    bestParamtes = pd.DataFrame(BoasEstratégiasParaSalvar)
    bestParamtes.to_csv("estrategy results/"+name+".csv", sep=';')


In [19]:
data = json.loads(requests.get(
    "https://api.binance.com/api/v1/exchangeInfo").text)
data = data['symbols']


In [20]:
pares = []
matches = [x for x in data if x['quoteAsset'] == "USDT"]
matches.sort
for matche in matches:
    if  not matche['symbol'] == "ADAUSDT" and not matche['symbol'] == "BNBUSDT" and not match['symbol'] == "BTCUSDT" and not match['symbol'] == "EOSUSDT" and not matche['symbol'] == "ETHUSDT"  and not matche['symbol'] == "IOTAUSDT"  and not matche['symbol'] == "LTCUSDT"  and not matche['symbol'] == "NEOUSDT"  and not matche['symbol'] == "ONTUSDT"  and not matche['symbol'] == "QTUMUSDT"  and not matche['symbol'] == "TUSDUSDT"  and not matche['symbol'] == "XLMUSDT"  and not matche['symbol'] == "XRPUSDT"
        print('\n', matche['symbol'])
        dataframe = get_bars(matche['symbol'], "1w")
        filename = matche['symbol']+"_"+"1w_SmaCross_Strategy"
        gerarParametros(dataframe, filename, SmaCross, 100)



 BNBUSDT
https://api.binance.com/api/v1/klines?symbol=BNBUSDT&interval=1w&limit=1001

testando nos valores  1 , 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100
testando nos valores  2 , 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100
testando nos valores  3 , 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16

/home/pedro/anaconda3/lib/python3.7/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100
testando nos valores  14 , 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100
testando nos valores  15 , 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65

/home/pedro/anaconda3/lib/python3.7/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100
testando nos valores  72 , 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100
testando nos valores  73 , 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 4

...

/home/pedro/anaconda3/lib/python3.7/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100
testando nos valores  44 , 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100
testando nos valores  45 , 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70

/home/pedro/anaconda3/lib/python3.7/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100
testando nos valores  55 , 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100
testando nos valores  56 , 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68

KeyboardInterrupt: 

In [ ]:
# print(maiorRetorno)
# print(maoirAcerto)
# print(BoasEstratégiasParaSalvar)
# gerarParametros()
# bestParamtes = pd.DataFrame (BoasEstratégiasParaSalvar)
# bestParamtes.to_csv("estrategy results/melhores parametros.csv", sep=';')


# Nova seção


In [ ]:

# SmaCross.setInitialParams(SmaCross,12, 15)
# bt = Backtest(GOOG, SmaCross,cash=1000,exclusive_orders=True)
# output = bt.run()
# print(output)
# SmaCross.setInitialParams(SmaCross,29, 30)
# bt = Backtest(dataframe, SmaCross,cash=1000000)
# output = bt.run()
# output
# bt.plot()
